In [1]:
import sqlite3
import cv2
from pyzbar.pyzbar import decode
import numpy as np
import pandas as pd
from collections import deque

In [ ]:
# カメラを初期化（カメラデバイスID 0を指定）
cap = cv2.VideoCapture(0)

# カメラの初期化が成功したか確認
if not cap.isOpened():
    print("カメラが見つかりません")
    exit()
W, H = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# 高解像度に設定
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)  # カメラの実際の解像度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H) # カメラの実際の解像度    
# 
# scale = 1
# 
# # ウィンドウサイズのピクセル数を算出
# width_pixels = int(W * scale)
# height_pixels = int(H * scale)
# 
# # リサイズ可能なウィンドウを作成し、物理サイズに近い表示サイズに設定
# cv2.namedWindow('Camera', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Camera', height_pixels, width_pixels)

# リアルタイム表示
while True:
    # カメラからフレームを読み込む
    ret, frame = cap.read()

    # フレームの読み込みが成功したか確認
    if not ret:
        print("フレームを読み込めません")
        break

    # フレームを表示
    cv2.imshow('Camera', frame)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# カメラリソースを解放し、ウィンドウを閉じる
cap.release()
cv2.destroyAllWindows()


In [2]:
path_to_db = f"C:\\Users\\tora2\\IdeaProjects\\cityScope\\visual\\tangible_ui_01\\tile.db"
NO_OF_CAMERA = 0

In [3]:
#投影変換座標取得
def setting_sikaku():
    # カメラの初期化 (0はデフォルトのカメラ)
    cap = cv2.VideoCapture(NO_OF_CAMERA)
    
    # グローバル変数として使用
    points = []
    # マウスコールバック関数
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN and len(points) < 4:
            # リサイズ後のクリック座標を元の画像の座標に変換
            points.append((x, y))
            print(f"選択した点: {len(points)}: ({x}, {y})")
            if len(points) == 4:
                cv2.destroyWindow("Select Points")
    # リアルタイムでカメラ映像を表示
    print("任意のキーを押して写真を撮影してください。")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("カメラから画像を取得できませんでした。")
            break
        # カメラ映像を表示
        cv2.imshow("Webcam - Press any key to capture", frame)
        # 'q'キーで撮影せずに終了
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("終了します。")
            cap.release()
            cv2.destroyAllWindows()
            exit()
        # 任意のキーを押すと撮影
        if cv2.waitKey(1) != -1:
            print("写真を撮影しました。")
            break
    # マウスで4点を選択
    print("左上、右上、右下、左下の順で4つの点をクリックしてください。")
    cv2.imshow("Select Points", frame)
    cv2.setMouseCallback("Select Points", select_point)
    # 4点が選ばれるまで待機
    while len(points) < 4:
        cv2.imshow("Select Points", frame)
        cv2.waitKey(1)
    # 選択された4つの点を取得
    top_left, top_right, bottom_right, bottom_left = points
    # 座標の表示
    print(f"左上: {top_left}")
    print(f"右上: {top_right}")
    print(f"右下: {bottom_right}")
    print(f"左下: {bottom_left}")
    # 選択したポイントに色付き円を描画して確認
    cv2.circle(frame, top_left, 5, (0, 255, 0), -1)
    cv2.circle(frame, top_right, 5, (255, 0, 0), -1)
    cv2.circle(frame, bottom_right, 5, (255, 255, 0), -1)
    cv2.circle(frame, bottom_left, 5, (0, 0, 255), -1)
    # 結果を表示
    cv2.imshow("Selected Points", frame)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # リソースの解放
    cap.release()
    return top_left, top_right, bottom_right, bottom_left
# 透視変換関数の定義
def PerspectiveTransform_2(image, W, H, tl, tr, br, bl):
    # 視点の表裏を逆転
    src_pts = np.array([tr, tl, bl, br], dtype='float32')
    dst_pts = np.array([(0, 0), (W, 0), (W, H), (0, H)], dtype='float32')
    # 変換行列を取得し変換を適用
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (W, H))
    return warped
# グリッドセルを計算する関数（右方向がX軸、下方向がY軸）
def find_grid_cell(x, y, W, H, N, L):
    # グリッドの各セルの幅と高さを計算
    cell_width = W / L
    cell_height = H / N
    # 横方向（X軸）と縦方向（Y軸）のインデックスを計算
    col_index = int(x // cell_width)  # X軸方向のインデックス（右方向）
    row_index = int(y // cell_height) # Y軸方向のインデックス（下方向）
    # インデックスが範囲外にならないように調整
    col_index = min(col_index, L - 1)
    row_index = min(row_index, N - 1)
    return col_index,row_index
# QRコードの中心座標を取得してグリッドを表示する関数
def detect_qr(frame, W, H, N, L):
    # 初期状態のデータフレーム
    grid_data = pd.DataFrame([(x, y, 0, None) for y in range(N) for x in range(L)],
                             columns=['x_rel', 'y_rel', 'flag', 'barcode_type'])
    # QRコードを検出
    barcodes = decode(frame)
    # 各QRコードについて
    for barcode in barcodes:
        # QRコードの位置を取得
        (x, y, w, h) = barcode.rect
        # 中心座標を計算
        center_x = x + w // 2
        center_y = y + h // 2
        center = (center_x, center_y)
        # 中心座標からグリッド位置を取得
        grid_x, grid_y = find_grid_cell(center_x, center_y, W, H, N, L)
        # QRコードの内容をデコード
        barcode_data = barcode.data.decode("utf-8")
        barcode_type = barcode.type
        # データフレームの該当セルを更新
        grid_data.loc[(grid_data['x_rel'] == grid_x) & (grid_data['y_rel'] == grid_y), 'flag'] = 1
        grid_data.loc[(grid_data['x_rel'] == grid_x) & (grid_data['y_rel'] == grid_y), 'barcode_type'] = barcode_type
        # 中心座標とグリッド位置を画像に描画
        cv2.circle(frame, center, 5, (0, 255, 0), -1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = f"Grid: ({grid_x}, {grid_y})"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # 結果の画像を表示

    cv2.imshow("QR Code Detection", frame)
    # 履歴に追加
    history.append(grid_data)

    return aggregate_qr_state(history, N, L)
# 過去Nフレームの状態から集計する関数
def aggregate_qr_state(history, N, L):
    # デフォルトの集計用データフレームを作成
    agg_data = pd.DataFrame([(x, y, 0, None) for y in range(N) for x in range(L)],
                            columns=['x_rel', 'y_rel', 'flag', 'barcode_type'])
    # 各グリッドごとにQRコードの状態を集計
    for (x, y) in zip(agg_data['x_rel'], agg_data['y_rel']):
        # 過去のデータから該当するグリッドの `flag` と `barcode_type` のリストを作成
        flag_list = [df.loc[(df['x_rel'] == x) & (df['y_rel'] == y), 'flag'].values[0] for df in history]
        barcode_type_list = [df.loc[(df['x_rel'] == x) & (df['y_rel'] == y), 'barcode_type'].values[0] for df in history]

        # 過去Nフレームで一度でも `flag` が1なら存在すると判断
        if any(flag_list):
            agg_data.loc[(agg_data['x_rel'] == x) & (agg_data['y_rel'] == y), 'flag'] = 1
            # 存在する場合の `barcode_type` を設定（最頻値を保持）
            most_common_type = max(set(barcode_type_list), key=barcode_type_list.count)
            agg_data.loc[(agg_data['x_rel'] == x) & (agg_data['y_rel'] == y), 'barcode_type'] = most_common_type

    return agg_data

def update(data):
    # データベースに接続
    conn = sqlite3.connect(path_to_db)
    cursor = conn.cursor()
    for x,y,l in zip(data.x_rel,data.y_rel,data.flag):
        label = 0
        if l == 1:
            label = 2

        cursor.execute("""
            UPDATE tile
            SET label = ?
            WHERE x_rel = ? AND y_rel = ?
        """, (label,x,y))

    # 変更をコミットして、データベース接続を閉じる
    conn.commit()
    conn.close()

In [4]:
N = 10  # 過去Nフレーム分を蓄積する
history = deque(maxlen=N)
# コントラストと明るさの変更
alpha = 1.5  # コントラストの倍率（1より大きい値でコントラストが上がる）
beta = 0  # 明るさの調整値（正の値で明るくなる）

# カメラの設定
cap = cv2.VideoCapture(NO_OF_CAMERA)

# 高解像度に設定
W,H = cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(f"W:{W} H:{H}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)  # カメラの実際の解像度
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H) # カメラの実際の解像度 

# リサイズ可能なウィンドウを作成し、サイズを指定
cv2.namedWindow("QR Code Detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow("QR Code Detection", 1290, 1290)

if not cap.isOpened():
    print("カメラが見つかりませんでした。")
    exit()
# グリッドのサイズと透視変換後のサイズを設定
W, H = 1000, 1000  # 透視変換後の幅と高さ
N, L = 10, 10  # グリッドの行数と列数
# 透視変換のための四隅の座標を取得
top_left, top_right, bottom_right, bottom_left = setting_sikaku()
data_list=[]
# フレーム処理のループ
while True:
    # time.sleep(0.1)
    ret, frame = cap.read()
    if not ret:
        
        print("カメラから画像を取得できませんでした。")
        break
    # 透視変換を適用
    perce_frame = PerspectiveTransform_2(frame, W, H, top_left, top_right, bottom_right, bottom_left)
    adjusted_frame = cv2.convertScaleAbs(perce_frame, alpha=alpha, beta=beta)
    data=detect_qr(adjusted_frame, W, H, N, L)
    update(data)

    # 'ESC'キーで終了
    if cv2.waitKey(1) & 0xFF == 27:
        break
# カメラとウィンドウのリソース解放
cap.release()
cv2.destroyAllWindows()

W:1280.0 H:720.0
任意のキーを押して写真を撮影してください。
写真を撮影しました。
左上、右上、右下、左下の順で4つの点をクリックしてください。
選択した点: 1: (360, 56)
選択した点: 2: (1001, 30)
選択した点: 3: (1024, 676)
選択した点: 4: (370, 684)
左上: (360, 56)
右上: (1001, 30)
右下: (1024, 676)
左下: (370, 684)


In [9]:
# パラメータの設定
x_value = 3
y_value = 2
label_value = 0

# データベースに接続
conn = sqlite3.connect(path_to_db)
cursor = conn.cursor()

# パラメータを使用して更新クエリを実行
cursor.execute("""
    UPDATE tile
    SET label = ?
    WHERE x_rel = ? AND y_rel = ?
""", (label_value, x_value, y_value))

# 変更をコミットして、データベース接続を閉じる
conn.commit()
conn.close()
